In [1]:
!pip install pymprog

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 26.7 MB/s eta 0:00:00
  Created wheel for pymprog: filename=pymprog-1.1.2-py3-none-any.whl size=43485 sha256=b74db12b71c6b756b390ecd4509239c75c668ffcf727a883ff763775e053a370
  Stored in directory: /root/.cache/pip/wheels/f7/dc/68/18b668a7137dc2a1e5973c9c3c8543ddcce4daa09a3a8d94b0
Successfully built pymprog


In [3]:
from pymprog import*
from math import*
#
import folium

coordenadas = [["Ribeirão Preto",-21.151998, -47.796710],
               ["Cravinhos",-21.342475, -47.736670],
               ["Sertaozinho",-21.137034, -47.995192],
               ["Jaboticabal",-21.2554, -48.3224],
               ["Pitangueiras",-21.012735, -48.217665],
               ["Batatais",-20.885360, -47.582476],
               ["Ribeirão Preto",-21.151998, -47.796710],]
               
centro = [-21.151998, -47.796710] # ponto central para plotar o mapa
n = len(coordenadas) # número de cidades (nós)

begin('PCV')

x = var('x',iprod(range(n),range(n)),bool)
u = var('u', n) #variável continua da ordenação


# função objetivo
minimize(sum(sqrt((coordenadas[i][1]-coordenadas[j][1])**2 + (coordenadas[i][2]-coordenadas[j][2])**2)*x[i,j] for i in range(n-1) for j in range(1,n) if i!=j))

# primeira restrição
for i in range(n-1):
  sum(x[i,j] for j in range(1,n) if i!=j) == 1

# segunda restrição
for j in range(1,n):
  sum(x[i,j] for i in range(n-1) if i!=j) == 1

# restrinção do fi que 'ordena' a rota corretamente e automaticamente remove sub-rotas
for i in range(n-1):
  for j in range(1,n):
    u[j] >= u[i] + n * x[i,j] - n + 1

#solver(int, gmi_cuts=1)
solve()

print("Valor ótimo PCV = {:.3f}".format(vobj()))
print()
i = 0
j = 1
cont = 0
while cont < n-1 and i != n-1:
  while x[i,j].primal < 0.9:
    j = j+1
  print("{:15s} -->    {:15s}".format(coordenadas[i][0],coordenadas[j][0]))
  i = j
  j = 1
  cont = cont + 1


# criação e impressão do mapa
arcos = []
for i in range(n):
  for j in range(n):
    if i != j:
      if x[i,j].primal > 0.1:
        arcos.append([i,j])




map = folium.Map(location=centro, zoom_start=8.5)
for arco in arcos:
  folium.PolyLine([[  coordenadas[arco[0]][1] , coordenadas[arco[0]][2]],[coordenadas[arco[1]][1] , coordenadas[arco[1]][2]]]).add_to(map)
for local in coordenadas:
  folium.Marker(
      location=[local[1],local[2]],
    popup= local[0]).add_to(map)
display(map)

end()

Valor ótimo PCV = 2.132

Ribeirão Preto  -->    Cravinhos      
Cravinhos       -->    Sertaozinho    
Sertaozinho     -->    Jaboticabal    
Jaboticabal     -->    Pitangueiras   
Pitangueiras    -->    Batatais       
Batatais        -->    Ribeirão Preto 


model('PCV') is not the default model.